In [ ]:
pip install haversine

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
%matplotlib inline

from haversine import haversine, Unit

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_place = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/place.xlsx')
df_bus = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/bus.xlsx')
df_subway = pd.read_excel('/content/drive/MyDrive/Colab Notebooks/subway.xlsx')

In [ ]:
df_place.head()

,명소 ID,키워드,명칭,시/도,시/군/구,동/면/읍/리,전체 주소,명소_위도,명소_경도,지하철역,지하철역_위도,지하철역_경도,지하철역_거리,버스정류장,버스정류장_위도,버스정류장_경도,버스정류장_거리
0,1.0,관광 명소,63스퀘어,서울시,영등포구,여의도동,서울특별시 영등포구 여의도동 60,37.51957,126.9398,여의나루역,37.527098,126.932901,NaN,NaN,NaN,NaN,NaN
1,2.0,관광 명소,몽촌역사관,서울시,송파구,방이동,서울특별시 송파구 방이동 88-3,37.52442,127.1232,올림픽공원역,37.516078,127.130848,NaN,NaN,NaN,NaN,NaN
2,3.0,관광 명소,경복궁,서울시,종로구,세종로,서울특별시 종로구 세종로 1-1,37.57764,126.9780,경복궁역,37.575762,126.973530,NaN,NaN,NaN,NaN,NaN
3,4.0,관광 명소,경희대학교 자연사박물관,서울시,동대문구,회기동,서울특별시 동대문구 회기동 1,37.59207,127.0555,외대앞역,37.596073,127.063549,NaN,NaN,NaN,NaN,NaN
4,5.0,관광 명소,광화문광장,서울시,종로구,세종로,서울특별시 종로구 세종로 1-68,37.57243,126.9769,경복궁역,37.575762,126.973530,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_subway.head()

,역ID,지하철역,지하철역_위도,지하철역_경도
0,1.0,가락시장,37.492522,127.118234
1,2.0,종로3가,37.571607,126.991806
2,3.0,오금,37.502162,127.128111
3,4.0,동대문,37.571420,127.009745
4,5.0,동대문역사문화공원,37.565138,127.007896


In [ ]:
df_bus.head()

,정류장ID,정류소번호,정류소명,X좌표,Y좌표
0,1.0,3007.0,삼각지역,126.974012,37.535270
1,2.0,4843.0,옥수교회,127.015895,37.545305
2,3.0,5120.0,면곡시장,127.079729,37.569746
3,4.0,8186.0,월계2교,127.050133,37.623825
4,5.0,8901.0,수보약국,127.008044,37.608704


# **Input Feature Optimization**

In [ ]:
# ① 명소 data set에서 좌표 위치만 남기고 나머지 column들은 drop
place_cord = df_place.drop(['키워드', '명칭', '시/도', '시/군/구', '동/면/읍/리', '전체 주소'],1)
place_cord.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  


,명소 ID,명소_위도,명소_경도,지하철역,지하철역_위도,지하철역_경도,지하철역_거리,버스정류장,버스정류장_위도,버스정류장_경도,버스정류장_거리
0,1.0,37.51957,126.9398,여의나루역,37.527098,126.932901,NaN,NaN,NaN,NaN,NaN
1,2.0,37.52442,127.1232,올림픽공원역,37.516078,127.130848,NaN,NaN,NaN,NaN,NaN
2,3.0,37.57764,126.9780,경복궁역,37.575762,126.973530,NaN,NaN,NaN,NaN,NaN
3,4.0,37.59207,127.0555,외대앞역,37.596073,127.063549,NaN,NaN,NaN,NaN,NaN
4,5.0,37.57243,126.9769,경복궁역,37.575762,126.973530,NaN,NaN,NaN,NaN,NaN


In [ ]:
# ② 명소 위치 ~ 지하철역 위치 중 가장 최솟값인 좌표 + 역명 + 거리(km) 계산/선택하여 append
station_name = ''
shortest_x = 0
shortest_y = 0
shortest_dist = 0

for a in range(max(place_cord['명소 ID'].astype(int))):
  for b in range(max((df_subway['역ID'].astype(int)))):
    place = (place_cord.iloc[a,1], place_cord.iloc[a,2])
    subway = (df_subway.iloc[b,2], df_subway.iloc[b,3])
    dist_temp = haversine(place, subway, unit='km')
    
    if b == 0:
      station_name = df_subway.iloc[b,1]
      shortest_x = df_subway.iloc[b,2]
      shortest_y = df_subway.iloc[b,3]
      shortest_dist = dist_temp
    else:
      subway_before = (shortest_x, shortest_y)
      if dist_temp < haversine(place, subway_before, unit='km'):
        station_name = df_subway.iloc[b,1]
        shortest_x = df_subway.iloc[b,2]
        shortest_y = df_subway.iloc[b,3]
        shortest_dist = dist_temp
  
  place_cord.iloc[a,3] = station_name
  place_cord.iloc[a,4] = shortest_x
  place_cord.iloc[a,5] = shortest_y
  place_cord.iloc[a,6] = shortest_dist

place_cord.head()

,명소 ID,명소_위도,명소_경도,지하철역,지하철역_위도,지하철역_경도,지하철역_거리,버스정류장,버스정류장_위도,버스정류장_경도,버스정류장_거리
0,1.0,37.51957,126.9398,노량진,37.514219,126.942454,0.639392,NaN,NaN,NaN,NaN
1,2.0,37.52442,127.1232,강동구청,37.530341,127.120508,0.699877,NaN,NaN,NaN,NaN
2,3.0,37.57764,126.9780,경복궁,37.575762,126.973530,0.445852,NaN,NaN,NaN,NaN
3,4.0,37.59207,127.0555,회기,37.589460,127.057583,0.343382,NaN,NaN,NaN,NaN
4,5.0,37.57243,126.9769,광화문,37.571026,126.976669,0.157440,NaN,NaN,NaN,NaN


In [ ]:
# ③ 명소 위치 ~ 버스정류장 위치 중 가장 최솟값인 좌표 + 정류소명 + 거리(km) 계산/선택하여 append
station_name = ''
shortest_x = 0
shortest_y = 0
shortest_dist = 0

for c in range(max(place_cord['명소 ID'].astype(int))):
  for d in range(max((df_bus['정류장ID'].astype(int)))):
    place = (place_cord.iloc[c,1], place_cord.iloc[c,2])
    bus = (df_bus.iloc[d,4], df_bus.iloc[d,3])
    dist_temp = haversine(place, bus, unit='km')
    
    if d == 0:
      station_name = df_bus.iloc[d,2]
      shortest_x = df_bus.iloc[d,4]
      shortest_y = df_bus.iloc[d,3]
      shortest_dist = dist_temp
    else:
      bus_before = (shortest_x, shortest_y)
      if dist_temp < haversine(place, bus_before, unit='km'):
        station_name = df_bus.iloc[d,2]
        shortest_x = df_bus.iloc[d,4]
        shortest_y = df_bus.iloc[d,3]
        shortest_dist = dist_temp
  
  place_cord.iloc[c,7] = station_name
  place_cord.iloc[c,8] = shortest_x
  place_cord.iloc[c,9] = shortest_y
  place_cord.iloc[c,10] = shortest_dist

place_cord.head()

,명소 ID,명소_위도,명소_경도,지하철역,지하철역_위도,지하철역_경도,지하철역_거리,버스정류장,버스정류장_위도,버스정류장_경도,버스정류장_거리
0,1.0,37.51957,126.9398,노량진,37.514219,126.942454,0.639392,63빌딩.가톨릭대학교여의도성모병원,37.519162,126.937709,0.189894
1,2.0,37.52442,127.1232,강동구청,37.530341,127.120508,0.699877,현대토파즈아파트,37.525305,127.125861,0.254486
2,3.0,37.57764,126.9780,경복궁,37.575762,126.973530,0.445852,경복궁,37.576950,126.979338,0.140649
3,4.0,37.59207,127.0555,회기,37.589460,127.057583,0.343382,회기시장,37.591203,127.056639,0.139184
4,5.0,37.57243,126.9769,광화문,37.571026,126.976669,0.157440,KT광화문지사,37.572226,126.977332,0.044298


In [ ]:
# ④ 기존의 DataFrame으로 삽입 후 Excel 파일로 저장
df_place['버스정류장'] = place_cord['버스정류장']     # Bus에 대한 data update
df_place['버스정류장_위도'] = place_cord['버스정류장_위도']
df_place['버스정류장_경도'] = place_cord['버스정류장_경도']
df_place['버스정류장_거리'] = place_cord['버스정류장_거리']

df_place['지하철역'] = place_cord['지하철역']         # Subway에 대한 data update
df_place['지하철역_위도'] = place_cord['지하철역_위도']
df_place['지하철역_경도'] = place_cord['지하철역_경도']
df_place['지하철역_거리'] = place_cord['지하철역_거리']

df_place.to_excel('/content/drive/MyDrive/Colab Notebooks/place_complete.xlsx', index=False)